# Principais métodos de chains com LCEL

A interface padrão de LCEL inclui os seguintes métodos:

- **stream:** transmitir de volta fragmentos da resposta
- **invoke:** chamar a cadeia com um input
- **batch:** chamar a cadeia com uma lista de inputs

Esses também possuem métodos assíncronos correspondentes que devem ser usados com a sintaxe `asyncio await` para concorrência:

- **astream:** transmitir de volta fragmentos da resposta de forma assíncrona
- **ainvoke:** chamar a cadeia com um input de forma assíncrona
- **abatch:** chamar a cadeia com uma lista de inputs de forma assíncrona
- **astream_log:** transmitir de volta etapas intermediárias à medida que acontecem, além da resposta final
- **astream_events:** transmitir eventos beta à medida que acontecem na cadeia

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(model="gpt-3.5-turbo")
prompt = ChatPromptTemplate.from_template("Crie uma frase sobre o assunto: {assunto}")

chain = prompt | model

## Invoke

O invoke é o método básico para inserir uma input na cadeia e receber uma resposta

In [3]:
chain.invoke({'assunto': 'cachorrinhos'})

AIMessage(content='"Cachorrinhos são a melhor companhia para alegrar nossos dias com sua lealdade e carinho incondicional."', response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 20, 'total_tokens': 50}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-feb1cf28-b951-4692-8582-eb8eaab16046-0')

Ele pode ser rodado como uma simples string quando existe apenas uma input no prompt, mas a forma mais recomendada é informando especificamente o nome da input através de um dicionário.

In [5]:
chain.invoke('cachorrinhos')

AIMessage(content='Cachorrinhos são companheiros leais que enchem nossas vidas de alegria e amor incondicional.', response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 20, 'total_tokens': 48}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e83cee0e-4558-4198-8270-cd6cd4d52f6c-0')

## Stream

Para recebermos uma saída conforme ela é gerada pelo modelo utilizamos o stream


In [8]:
for chunck in chain.stream({'assunto': 'cachorrinho'}):
    print(chunck.content, end='', flush=True)

O cachorrinho é o melhor amigo do homem, sempre fiel e carinhoso.

## Batch

Para fazermos múltimpas requisições em paralelo utilizamos o batch

In [9]:
chain.batch([{'assunto': 'cachorrinhos'}, {'assunto': 'gatinho'}, {'assunto': 'cavalinhos'}])

[AIMessage(content='Cachorrinhos são alegria em quatro patas, sempre prontos para nos alegrar com suas brincadeiras e lealdade incondicional.', response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 20, 'total_tokens': 58}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-74619b29-27be-45b2-a0e2-e6684353dc9a-0'),
 AIMessage(content='"Um gatinho é o melhor amigo que você pode ter, sempre pronto para te fazer companhia e te encher de amor e carinho."', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 19, 'total_tokens': 51}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9538d682-6d94-4c38-ae1c-5dfd880ae4af-0'),
 AIMessage(content='Os cavalinhos de carrossel sempre trazem alegria e nostalgia para quem se aventura a dar uma volta neles.', response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 1

In [10]:
chain.batch([{'assunto': 'cachorrinhos'}, {'assunto': 'gatinho'}, {'assunto': 'cavalinhos'}], config={'max_concurrency': 5})

[AIMessage(content='Cachorrinhos são alegria em forma de patinhas e pelagem fofa, trazendo amor e companheirismo para nossas vidas.', response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 20, 'total_tokens': 57}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f705e146-799f-414c-ba4d-8c1ad84a3bc0-0'),
 AIMessage(content='O gatinho é a pura definição de fofura e alegria em forma de pelagem.', response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 19, 'total_tokens': 44}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-dbdf6dd8-87b7-431d-8e25-9a7ffaf1dfc2-0'),
 AIMessage(content='Os cavalinhos de carrossel sempre trazem alegria e diversão para as crianças.', response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 19, 'total_tokens': 42}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'fi

## Ainvoke

Todos os métodos possuem assíncronos correspondentes que devem ser usados com a sintaxe `asyncio await` para concorrência

In [12]:
import asyncio

async def processa_chain(input):
    resposta = await chain.ainvoke(input)
    return resposta

In [15]:
task1 = asyncio.create_task(processa_chain({'assunto': 'gatinho'}))
task2 = asyncio.create_task(processa_chain({'assunto': 'cavalinhos'}))
task3 = asyncio.create_task(processa_chain({'assunto': 'cachorrinhos'}))

await task1
await task2
await task3

AIMessage(content='Cachorrinhos são a alegria em quatro patas que iluminam nossas vidas com amor incondicional e fofura infinita.', response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 20, 'total_tokens': 55}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6c3b6ed5-5013-48e4-97d1-b188f8412848-0')

In [18]:
task2.result()

AIMessage(content='Os cavalinhos de carrossel são símbolos de infância e diversão que nos transportam para um mundo mágico de fantasia.', response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 19, 'total_tokens': 52}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b45432e2-ae39-4bf4-bc7a-2a89b03393ae-0')

### Calculando a diferença de tempo ao utilizar async

In [24]:
from time import time

a = time()

task1 = asyncio.create_task(processa_chain({'assunto': 'gatinho'}))
task2 = asyncio.create_task(processa_chain({'assunto': 'cavalinhos'}))
task3 = asyncio.create_task(processa_chain({'assunto': 'cachorrinhos'}))

await task1
await task2
await task3

b = time()
print(b-a)

1.2480289936065674


In [27]:
from time import time

a = time()

task1 = chain.invoke({'assunto': 'gatinho'})
task2 = chain.invoke({'assunto': 'cavalinhos'})
task3 = chain.invoke({'assunto': 'cachorrinhos'})

b = time()
print(b-a)

3.2890777587890625
